<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.977 · Anàlisi de sentiments i textos</p>
<p style="margin: 0; text-align:right;">Màster universitari en Ciències de dades (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudi d'Informàtica, Multimèdia i Telecomunicacions</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>

# PRA 3: Deep Learning per a l'anàlisi de textos i sentiments

En aquesta pràctica revisarem i aplicarem els coneixements apresos en els mòduls 4 i 5. Tractarem els següents temes:

1. ** Traducció automàtica **: amb custom embeddings i amb embeddings preentrenats.
2. ** Classificació de frases **: Aplicació dels conceptes treballats per a la reutilització de l'arquitectura de dos models.
3. ** Anàlisi de sentiments **: anàlisi de sentiments de textos.
4. ** Anàlisi de sentiments per aspectes (ABSA) **: anàlisi d'aspectes en comentaris.




Així com altres temes transversals treballats al llarg de l'assignatura.

# PART 1

En aquesta primera part de la pràctica es demana resoldre els exercici amb la llibreria **Keras**.

# 1. Traducció Automàtica (3 punts)

## 1.1 TA amb Custom Embeddings (2 punts)


L'objectiu d'aquest apartat és entrenar un model de traducció automàtica entre anglès i alemany, seguint els mateixos passos que en el notebook de Machine Translation del mòdul 4.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Implementació:</strong> Seguint els passos treballats en el notebook de traducció automàtica, i partint de les mateixes dades, implementa i entrena un model de traducció automàtica, aquesta vegada de l'anglès a l'alemany. <br>
    - La capa embedding ha de tenir dimensió igual a 300 <br>
    - es recomana una longitud màxima de seqüència de 12 <br>
    - es recomana usar els primers 50000 parells de el corpus deu.txt <br>
    
Mostra l'aplicació del model entrenat amb algun exemple.
</div>

In [1]:
#############################################
# SOLUCIÓ                                   #
#############################################
from numpy import array

# Lectura del text
def read_text(filename):
        file = open(filename, mode='rt', encoding='utf-8')
        text = file.read()
        file.close()
        return text

# Text a parells angles-alemany
def to_lines(text):
    sents = text.strip().split('\n')
    sents = [i.split('\t')[:2] for i in sents]
    return sents    

data = read_text("deu.txt")
eng_deu = to_lines(data)
eng_deu = array(eng_deu)

eng_deu = eng_deu[:50000,:]

In [2]:
# Preprocessament
import string

# sense signes de puntuació
eng_deu[:,0] = [s.translate(str.maketrans('', '', string.punctuation)) for s in eng_deu[:,0]]
eng_deu[:,1] = [s.translate(str.maketrans('', '', string.punctuation)) for s in eng_deu[:,1]]

# tot en minúscules
for i in range(len(eng_deu)):
    eng_deu[i,0] = eng_deu[i,0].lower()
    eng_deu[i,1] = eng_deu[i,1].lower()

print(eng_deu)

[['hi' 'hallo']
 ['hi' 'grüß gott']
 ['run' 'lauf']
 ...
 ['i wholeheartedly agree' 'ich stimme rückhaltlos zu']
 ['i will always love you' 'ich werde dich immer lieben']
 ['i will be back by nine' 'um neun bin ich wieder zurück']]


In [3]:
# Creació del vocabulari amb parelles paraula-index
from keras.preprocessing.text import Tokenizer

def tokenization(sentences):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences)
    return tokenizer

engtok= tokenization(eng_deu[:,0])
deutok= tokenization(eng_deu[:,1])

engvsize= len(engtok.word_index)+1
deuvsize= len(deutok.word_index)+1
print('English Vocabulary Size: %d'%engvsize)
print('Deutch Vocabulary Size: %d'%deuvsize)

Using TensorFlow backend.


English Vocabulary Size: 6361
Deutch Vocabulary Size: 10597


In [4]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

# Conjunts de train i test
train, test = train_test_split(eng_deu, test_size=0.2, random_state=12)

# Codificació de les seqüencies amb els index de les paraules, i padding fins longitud
def encode_sequences(tokenizer, length, lines):
    seq = tokenizer.texts_to_sequences(lines)
    seq = pad_sequences(seq, maxlen=length, padding='post')
    return seq

seql=12
trainX = encode_sequences(engtok, seql, train[:,0])
trainY = encode_sequences(deutok, seql, train[:,1])
testX = encode_sequences(engtok, seql, test[:,0])
testY = encode_sequences(deutok, seql, test[:,1])

In [5]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector

# El model és sequencial i consta d'un encoder i un decoder
def encodeco(in_vocab_size, vec_length, max_text_length, out_timesteps, out_vocab_size):
    mt_model = Sequential()
    # Encoder
    # La capa embedding se inicializa con pesos aleatorios y aprende una representación vectorial (embeddings) 
    # para todas las palabras de las frases. Los embeddings podrian ser también vectores Word2Vec o Glove precalculados

    # Es necesario tener: 
    # el tamaño del vocabulario de los textos (vocab_size)
    # la dimensión de los vectores en los que las palabras serán embedded (embedding_vec_length).
    # la longitud máxima de las secuencias de input
    mt_model.add(Embedding(in_vocab_size, vec_length, input_length = max_text_length, mask_zero=True))
    mt_model.add(LSTM(vec_length))
    # Decoder
    #Repetición del último output tantas veces como la longitud máxima de la lengua destino (eng_length).
    #Fíjese que esta longitud máxima corresponde a estados de tiempo del output (out_timesteps)
    #https://stackoverflow.com/questions/51749404/how-to-connect-lstm-layers-in-keras-repeatvector-or-return-sequence-true
    mt_model.add(RepeatVector(out_timesteps))
    #Añadir la capa LSTM con el mismo número de units que el LSTM del codificador. Establecemos que queremos
    #obtener la secuencia completa del output; esto es, la traducción de la frase entera.
    mt_model.add(LSTM(vec_length, return_sequences=True))
    #Añadir la capa donde se representa la distribución de las palabras del vocabulario de la lengua destino 
    #según la probabilidad de aparición en la secuencia destino. Así podemos obtener la secuencia más probable
    #de ser la traducción de la frase origen
    mt_model.add(Dense(out_vocab_size, activation='softmax'))
    return mt_model

embedvl= 300
model  = encodeco(engvsize, embedvl, seql, seql, deuvsize)

In [6]:
from keras import optimizers
from keras.callbacks import ModelCheckpoint

# Compilació amb optimizador RMS
rms = optimizers.RMSprop(lr=0.001)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy')

filename = 'model_eng_deu'

#La función ModelCheckpoint() guarda el modelo con la pérdida de validación más baja.
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

#Entrenamiento del modelo. Se realiza con 30 epoch con un tamaño de batch de 256, con un reparto de validación del
#20%; esto es, 80% se destina al entrenamiento en sí, y el 20% restante a su validación.
model.fit(trainX, trainY.reshape(trainY.shape[0], trainY.shape[1], 1), epochs=30, batch_size=256,
          validation_split=0.2, callbacks=[checkpoint], verbose=0)

In [7]:
# Predicció
import pandas as pd
from keras.models import load_model

model = load_model('model_eng_deu')
preds = model.predict_classes(testX.reshape((testX.shape[0],testX.shape[1])))
print(preds)

def get_word(n, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == n:
            return word
    return None

preds_text = []
for i in preds:
    temp = []
    for j in range(len(i)):
        #Obtener la palabra que corresponde al índice del vocabulario de la lengua destino
        t = get_word(i[j], deutok)
        if j > 0:
            if (t == get_word(i[j-1], deutok)) or (t == None):
                     temp.append('')
            else:
                     temp.append(t)
        else:
            if(t == None):
                temp.append('')
            else:
                temp.append(t)
    preds_text.append(' '.join(temp).strip())

pred_df = pd.DataFrame({'actual' : test[:,1], 'predicted' : preds_text})
print(pred_df.iloc[100:120])

E:\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[[  8 199 199 ...   0   0   0]
 [  1  13  38 ...   0   0   0]
 [  1 195  15 ...   0   0   0]
 ...
 [  2   3  12 ...   0   0   0]
 [ 57  33   8 ...   0   0   0]
 [ 29  67  11 ...   0   0   0]]
                           actual                   predicted
100           stell das wasser ab             mach das wasser
101           sie sind gesprächig          du bist gesprächig
102         wer sind die typen da         wer ist diese leute
103           was würde passieren               was würde das
104                tom trägt blau                 tom hat den
105            sie sind glücklich          sie sind glücklich
106           eine frage habe ich         ich habe eine frage
107           sprich die wahrheit      sagen sie die wahrheit
108           das ist sehr salzig         das ist sehr dunkel
109         tom wirkt eingebildet   tom scheint unzuverlässig
110           ich muss tom warnen         ich muss tom finden
111        tom muss zu hause sein         tom muss nach hause
11

## 1.2 TA amb Embeddings preentrenats (1 punt)

En aquest apartat es demana repetir l'exercici anterior carregant a la capa d'embedding els pesos d'un model Glove entrenat per l'anglès.

Carreguem el següent model GloVe per l'anglès.

In [8]:
import numpy as np

embeddings_index = {}
f = open('glove.42B.300d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print(len(embeddings_index))

1917494


A continuació, hem de construir la matriu de embeddings.

Per no carregar tot el vocabulari de el model, hem de filtrar només aquelles entrades presents en el vocabulari del tokenizer que farem servir. I a més, hem d'incloure en la matriu vectors corresponidientes als índexs de les entrades (paraules) que no trobem en el model glove carregat. Aquests vectors es solen inicialitzar com 0s o com a resultats d'una distribució N (0,1),



Per exemple, si el nostre tokenizer es diu `eng_tokenizer` podríem fer:

In [9]:
embedding_matrix = np.zeros((len(engtok.word_index)+1, 300))
for word, i in engtok.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


Per iniciar una capa d'embedding amb pesos predefinits s'usa l'argument `weights`. A més, com no volem que es modifiquin els pesos vam marcar l'argument `trainable` com` False`.

Seguint amb el nostre exemple faríem:

In [10]:
embedding_layer = Embedding(len(engtok.word_index)+1,
                            embedvl,
                            weights=[embedding_matrix],
                            input_length=seql,
                            trainable=False,
                            mask_zero=True)

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Implementació:</strong> 
Implementa i entrena de nou un model de traducció automàtica de l'anglès a l'alemany de forma similar, aquest cop carregant els pesos de la capa embedding a partir d'el model Glove preentrat en anglès disponible a 'glove.42B.300d.txt'.
</div>

In [11]:
#############################################
# SOLUCIÓ                                   #
#############################################
def encodeco_glo(in_vocab_size, vec_length, max_text_length, out_timesteps, out_vocab_size):
    model = Sequential()
    model.add(embedding_layer)    # Embedding layer amb pesos predefinits
    model.add(LSTM(vec_length))
    model.add(RepeatVector(out_timesteps))
    model.add(LSTM(vec_length, return_sequences=True))
    model.add(Dense(out_vocab_size, activation='softmax'))
    return(model)

mod_glo= encodeco_glo(engvsize, embedvl, seql, seql, deuvsize)

In [12]:
rms = optimizers.RMSprop(lr=0.001)
mod_glo.compile(optimizer=rms, loss='sparse_categorical_crossentropy')
filename = 'model_eng_deu_glo'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
mod_glo.fit(trainX, trainY.reshape(trainY.shape[0], trainY.shape[1], 1), epochs=30, batch_size=256,
            validation_split=0.2, callbacks=[checkpoint], verbose=0)

In [13]:
model = load_model('model_eng_deu_glo')
preds = model.predict_classes(testX.reshape((testX.shape[0],testX.shape[1])))
print(preds)

def get_word(n, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == n:
            return word
    return None

preds_text = []
for i in preds:
    temp = []
    for j in range(len(i)):
        t = get_word(i[j], deutok)
        if j > 0:
            if (t == get_word(i[j-1], deutok)) or (t == None):
                     temp.append('')
            else:
                     temp.append(t)
        else:
            if(t == None):
                temp.append('')
            else:
                temp.append(t)
    preds_text.append(' '.join(temp).strip())

pred_df = pd.DataFrame({'actual' : test[:,1], 'predicted' : preds_text})
print(pred_df.iloc[100:120])

[[  8 199  27 ...   0   0   0]
 [  1  16  18 ...   0   0   0]
 [  2 195  15 ...   0   0   0]
 ...
 [  2   3  12 ...   0   0   0]
 [ 57  33   8 ...   0   0   0]
 [ 29  67  11 ...   0   0   0]]
                           actual              predicted
100           stell das wasser ab    mach das wasser aus
101           sie sind gesprächig                du bist
102         wer sind die typen da   wer sind diese leute
103           was würde passieren         was würde sich
104                tom trägt blau         tom trägt blau
105            sie sind glücklich     sie sind glücklich
106           eine frage habe ich    ich habe eine frage
107           sprich die wahrheit       sag sie wahrheit
108           das ist sehr salzig           das ist sehr
109         tom wirkt eingebildet      tom scheint nicht
110           ich muss tom warnen     ich muss tom sagen
111        tom muss zu hause sein       tom muss zu sein
112   tom verschränkte seine arme      tom hob seine arm
113       

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>(Opcional) Anàlisi:</strong> Explica quines són les principals diferències entre els dos models entrenats. Com podríem millorar els resultats d'aquesta tasca en concret?
</div>

- Tal com hem vist la diferència principal es troba en el fet de que la capa Embedding del model sigui supervisada o no. És a dir, si partim d'una matriu d'embedding aleatòria que haurà d'ajustar-se durant el entrenament (supervisada) o si comencem amb una matriu ja preentrenada amb capacitat d'extreure relacions semàntiques entre paraules.
- Pel que fa als resultats no sembla haver-hi un clarament millor que l'altre. El model preentrenat triga lleugerament menys en executar cada época del entrenament, fet coherent en tant que els pesos són fixes.
- Els punts que s'haurien de revisar a l'hora de considerar millores serien, per exemple, assegurar-nos de que el model supervisat tingui una quantitat de textos suficient i que el seu vocabulari sigui adient a l'estil del que volem traudir. També resultaria interessant fer servir tots dos métodes, generant la matriu d'embedding amb un preentrenament no-supervisat per després afinar els valors dels vectors durant l'entrenament de la xarxa.

# 2. Clasificació de frases (2 punts)

En aquest apartat plantegem l'ús de les arquitectures vistes fins al moment per millorar els resultats de la tasca de classificació d'opinions falses vista anteriorment en l'assignatura.


Primer carregarem les dades de el fitxer de titulars 'opinions-Tagged-FAKE.csv' amb l'objectiu d'entrenar un model que classifiqui les opinions a 'FAKENEG', 'FAKEPOS', 'TRUENEG' i 'TRUEPOS'.

In [14]:
import pandas as pd

df = pd.read_csv('OPINIONS-TAGGED-FAKE.csv', sep='\t')

data = []
data_labels = []

opinions = df['OPINION'].tolist()
tags = df['TAG'].tolist()

for i in range(len(opinions)): 
    data.append(opinions[i]) 
    data_labels.append(tags[i])


Preparem i preprocessem les dades per a l'entrenament. Farem servir one-hot encoding per a les etiquetes.

In [15]:
from numpy import array
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

values = array(data_labels)
print(values)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)

onehot_encoder = OneHotEncoder(categories='auto', sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

['FAKENEG' 'FAKENEG' 'FAKENEG' ... 'TRUEPOS' 'TRUEPOS' 'TRUEPOS']
[0 0 0 ... 3 3 3]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]



La idea d'el model de classificació que volem implementar és més simple que la de l'encoder-decoder usat en l'apartat 1.
El model ha de consistir només en:

- una capa embedding amb els pesos de el model Glove preentrenat per a l'anglès disponible en el fitxer 'glove.42B.300d.txt'
- una capa LSTM amb un nombre de units a triar (per exemple, 300)
- una capa Donin amb dimensió de sortida el nombre de categories amb les que volem classificar (en aquest cas, 4).

A més, com loss function `loss` farem servir 'categorical_crossentropy' i com` optimizer`, 'adam'.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong> Implementació:</strong> 
Adapta l'arquitectura vista en l'apartat anterior com s'indica a dalt per classificar els textos de opinions-Tagged-FAKED.csv en TRUENEG, FALSENEG, TRUEPOS, FALSEPOS. Compara els resultats amb els obtinguts en el Llibreta-PLA3.
</div>

In [16]:
#############################################
# SOLUCIÓ                                   #
#############################################
opin= [s.translate(str.maketrans('','',string.punctuation)).lower() for s in data]
for i in range(len(opin)):
    opin[i]= ''.join(opin[i]).split(' ')

# Tokenització
opitok= tokenization(opin)

# Matriu d'embedding preentrenada
embedding_matrix = np.zeros((len(opitok.word_index)+1, 300))
for word, i in opitok.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Split train/test i codificació de les sequencies
ml=0
for s in opin:
    if len(s)>ml:
        ml=len(s)

trainX, testX = train_test_split(opin, test_size=0.2, random_state=12)
codtrain, codtest = train_test_split(onehot_encoded, test_size=0.2, random_state=12)

opitrain= encode_sequences(opitok, ml, trainX)
opitest = encode_sequences(opitok, ml, testX)

# Definició i entrenament del model
modfake = Sequential()
modfake.add(Embedding(len(opitok.word_index)+1,
                      embedvl,
                      weights=[embedding_matrix],
                      input_length=ml,
                      trainable=False,
                      mask_zero=True))
modfake.add(LSTM(300))
modfake.add(Dense(len(set(values)), activation='softmax'))

filename = 'model_tag_fake'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

modfake.compile(optimizer='Adam', loss='categorical_crossentropy')
#modfake.fit(opitrain, codtrain, epochs=30, batch_size=256, validation_split=0.2,
#            callbacks=[checkpoint], verbose=1)

In [17]:
# Obtenció de mètriques
from sklearn import metrics

model = load_model('model_tag_fake')
preds = model.predict_classes(opitest.reshape((opitest.shape[0],opitest.shape[1])))

pred_df = pd.DataFrame({'actual' : np.argmax(codtest, axis=1), 'predicted' : preds})
print(metrics.classification_report(pred_df['actual'], pred_df['predicted']))
print(pred_df.sample(20))

              precision    recall  f1-score   support

           0       0.58      0.79      0.67        67
           1       0.79      0.81      0.80        77
           2       0.70      0.54      0.61        90
           3       0.76      0.71      0.73        86

    accuracy                           0.70       320
   macro avg       0.71      0.71      0.70       320
weighted avg       0.71      0.70      0.70       320

     actual  predicted
187       3          0
189       3          1
79        1          1
233       3          3
132       0          0
145       2          2
135       1          1
155       2          2
242       2          0
223       3          3
278       3          1
282       1          1
111       1          1
196       1          1
188       2          2
77        3          3
172       3          3
291       1          1
262       1          1
118       1          1


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong> Análisis:</strong>  
Segons el que hem estudiat a teoria, què podríem fer per trobar un model millor?
</div>

- Pel que fa als resultats respecte a PLA3, podem veure que els d'aquesta arquitectura només són superiors als de Random Forest. És a dir, qualsevol model purament matemàtic amb un classificador basat SVM o Regressió Logística aconseguiex resultats superiors a la xarxa neuronal recursiva.
- El resultat milloraria si el model fos capaç d'aplicar l'equivalent al sentit comú a les frases. Com a exemple tenim el model BERT, que permet fer una representació numèrica de les paraules segons el contexte en el que apareixen.

# PART 2

En aquesta segona part de la pràctica es demana resoldre els exercici amb la llibreria **PYTORCH**.

# 3. Anàlisi de sentiments (4 punts)


L'objectiu d'aquest apartat és entrenar un model d'anàlisi de sentiments per a un dataset de comentaris. El dataset consistirà en el camp `Short` de l'data frame de el fitxer 'tripadvisor_data.csv' i el sentiment el dóna el camp` Opinion`.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong> Implementació :</strong> 
Entrena un model d'anàlisi de sentiment que usi embeddings entrenats amb BERT seguint els mateixos passos que en primer notebook (SA) de la lliçó 5, ia partir de les dades de l'dataset de comentaris de camp `Short` per al text i del camp `Opinion` per al sentiment .
    
Compara els resultats amb els obtinguts a la PRA2.
</div>

In [18]:
#############################################
# SOLUCIÓ                                   #
#############################################
import pandas as pd
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

df = pd.read_csv('tripadvisor_data.csv')
print(df.head())

                                               Short  \
0                             “Very nice atmosphere”   
1  “Very nice food, great atmosphere, feels like ...   
2                         “Best Hotel on the Planet”   
3                        “What a vacation should be”   
4                                   “Excellent stay”   

                                                Long    Class Opinion  
0  We were together with some friends at the Anew...   family     POS  
1  Martin and his staff are truely great! They ma...   family     POS  
2  We have stayed at the Excelsior on numerous oc...   family     POS  
3  Having four days free in Milan, we decided to ...  friends     POS  
4  In all aspects an excellent stay. Professional...   couple     POS  


In [19]:
# model a partir d'embeddings BERT
# Lectura i generació de TEXT i LABEL
from transformers import BertModel, BertTokenizer
import random

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

def tokenize_and_cut(sentence):
    maxlen = tokenizer.max_model_input_sizes['bert-base-uncased']
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:maxlen-2]
    return tokens

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = tokenizer.cls_token_id,
                  eos_token = tokenizer.sep_token_id,
                  pad_token = tokenizer.pad_token_id,
                  unk_token = tokenizer.unk_token_id)
LABEL = data.LabelField(dtype = torch.float)

In [20]:
fields = [('text', TEXT),(None,None),(None,None),('label', LABEL)]

train_data = data.TabularDataset(path = 'tripadvisor_data.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True)

train_data, valid_data, test_data = train_data.split(split_ratio = [0.6, 0.2, 0.2], random_state = random.seed(SEED))

print(vars(train_data.examples[0]))
LABEL.build_vocab(train_data)
print(LABEL.vocab.freqs)

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_key = lambda x: len(x.text),
    sort_within_batch = False,
    device = device)

{'text': [1523, 2190, 2155, 3309, 1012, 2053, 2342, 2000, 3231, 2500, 2065, 2115, 4268, 2024, 3920, 2084, 2184, 1013, 2340, 1012, 1012, 1012, 1524], 'label': 'POS'}
Counter({'POS': 722, 'NEG': 155})


In [21]:
# Definició del model
import torch.nn as nn
import torch.optim as optim

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        self.bert = bert
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        with torch.no_grad(): # así congelamos los parámetros de BERT durante en entrenamiento
            embedded = bert(text)[0]
        
        _, hidden = self.rnn(embedded)
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        
        output = self.out(hidden)
        return output

In [22]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model     = model.to(device)
criterion = criterion.to(device)

#Número de parámetros
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'El modelo tiene {count_parameters(model):,} parámetros')

El modelo tiene 112,241,409 parámetros


In [23]:
# Entrenament del model
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:    
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [24]:
import time

def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    corrections = (rounded_preds == y).float()
    acc = corrections.sum() / len(corrections)
    return acc

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 4s
	Train Loss: 0.509 | Train Acc: 82.26%
	 Val. Loss: 0.500 |  Val. Acc: 79.06%
Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: 0.395 | Train Acc: 83.71%
	 Val. Loss: 0.439 |  Val. Acc: 81.63%
Epoch: 03 | Epoch Time: 0m 2s
	Train Loss: 0.310 | Train Acc: 88.56%
	 Val. Loss: 0.473 |  Val. Acc: 85.17%
Epoch: 04 | Epoch Time: 0m 2s
	Train Loss: 0.275 | Train Acc: 89.40%
	 Val. Loss: 0.448 |  Val. Acc: 85.42%
Epoch: 05 | Epoch Time: 0m 2s
	Train Loss: 0.246 | Train Acc: 91.31%
	 Val. Loss: 0.473 |  Val. Acc: 84.79%


In [25]:
#modificamos la función para que devuelva las predicciones
from sklearn import metrics

def evaluatemod(model, iterator, criterion):
    predictions_all = []
    labels_all = []
    
    model.eval()  # deshabilita dropout y batch normalization
    
    with torch.no_grad(): # para no calcular los gradientes durante las computaciones
    
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            predictions_all +=  torch.round(torch.sigmoid(predictions)).flatten().cpu().numpy().tolist()
            labels_all += batch.label.flatten().cpu().numpy().tolist()
        
    return predictions_all, labels_all

predictions, labels = evaluatemod(model, test_iterator, criterion)
print(metrics.classification_report(labels, predictions))

              precision    recall  f1-score   support

         0.0       0.87      0.96      0.91       238
         1.0       0.68      0.39      0.49        54

    accuracy                           0.85       292
   macro avg       0.78      0.67      0.70       292
weighted avg       0.84      0.85      0.84       292



- Podem veure que en aquest cas el resultat és similar al millor model obtingut en la PRA2, el TfIdf amb classificador SVM amb una accuracy de 0.89 i alta precisió a tots dos valors objectiu.

## Clasificación binaria de textos (1 punto)

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong> Implementació :</strong> 
Entenent la tasca d'anàlisi de sentiments com una classificació binari (p.ex. en 0 i 1), reutilitza la mateixa arquitectura per entrenar ara un classificador binari que classifiqui els titulars de camp 'Headline' a 'NYT-Comment-Headlines.csv' segons el camp 'Tag' a TOP i NOTOP.
    
Compara els resultats amb els obtinguts en la PRA1.
</div>

In [26]:
#############################################
# SOLUCIÓ                                   #
#############################################
fields = [(None,None),('text', TEXT),('label', LABEL)]

train_data = data.TabularDataset(path = 'NYT-Comment-Headlines.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True)

train_data, valid_data, test_data = train_data.split(split_ratio = [0.6, 0.2, 0.2], random_state = random.seed(SEED))

print(vars(train_data.examples[0]))
LABEL.build_vocab(train_data)
print(LABEL.vocab.freqs)

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_key = lambda x: len(x.text),
    sort_within_batch = False,
    device = device)

{'text': [2062, 6677, 2084, 18250, 1010, 1998, 2210, 3930, 2013, 9230], 'label': 'NONTOP'}
Counter({'NONTOP': 578, 'TOP': 253})


In [27]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut7-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: 0.675 | Train Acc: 64.01%
	 Val. Loss: 0.575 |  Val. Acc: 72.44%
Epoch: 02 | Epoch Time: 0m 2s
	Train Loss: 0.604 | Train Acc: 70.16%
	 Val. Loss: 0.549 |  Val. Acc: 73.07%
Epoch: 03 | Epoch Time: 0m 2s
	Train Loss: 0.535 | Train Acc: 74.00%
	 Val. Loss: 0.554 |  Val. Acc: 74.94%
Epoch: 04 | Epoch Time: 0m 2s
	Train Loss: 0.476 | Train Acc: 79.06%
	 Val. Loss: 0.551 |  Val. Acc: 75.57%
Epoch: 05 | Epoch Time: 0m 2s
	Train Loss: 0.435 | Train Acc: 80.98%
	 Val. Loss: 0.560 |  Val. Acc: 74.63%


In [28]:
predictions, labels = evaluatemod(model, test_iterator, criterion)
print(metrics.classification_report(labels, predictions))

              precision    recall  f1-score   support

         0.0       0.76      0.92      0.83       199
         1.0       0.56      0.24      0.34        78

    accuracy                           0.73       277
   macro avg       0.66      0.58      0.59       277
weighted avg       0.70      0.73      0.69       277



- El resultat és superior al Tfidf amb regressió lineal generat a la PRA1, el qual retornava un 0.71 de accuracy amb una precisió molt baixa pel que fa a la classificació de titulars TOP.

# 4. Anàlisi de sentiments per aspectes (1 punt)


En aquest apartat aplicarem els coneixements estudiats en la part d'ABSA del mòdul 5 i els combinarem amb altres coneixements per analitzar comentaris de tripadvisor sobre hotels.

Primer carreguem les dades per analitzar.

In [29]:
import pandas as pd
import numpy as np

tripadvisor_data = pd.read_csv('tripadvisor_data.csv')


A continuació carreguem un model d'anàlisi de sentiments per aspectes de Targeted ABSA.
L'etiqueta 0 correspon a negatiu, 1, a neutre i 2, a positiu.

In [30]:
from transformers import BertModel
import torch
import torch.nn as nn

BERT_DIM = 768
DROPOUT = 0.1
POLARITIES_DIM = 3

bert = BertModel.from_pretrained('bert-base-uncased')

class BERT_absa(nn.Module):
    def __init__(self, bert, dropout, bert_dim, polarities_dim):
        super(BERT_absa, self).__init__()
        self.bert = bert
        self.dropout = nn.Dropout(dropout)
        self.dense = nn.Linear(bert_dim, polarities_dim)

    def forward(self, inputs):
        text_bert_indices, bert_segments_ids = inputs.text, inputs.term
        _, pooled_output = self.bert(text_bert_indices, bert_segments_ids)
        pooled_output = self.dropout(pooled_output)
        logits = self.dense(pooled_output)
        return logits
    
modelba = BERT_absa(bert, DROPOUT, BERT_DIM, POLARITIES_DIM)

modelba.load_state_dict(torch.load('bert_absa'))
modelba.eval()

BERT_absa(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong> Implementació :</strong> 
Selecciona almenys dues parelles de `frases-terme` usant elements de camp 'Short' com` frase` i termes de la mateixa frase que siguin aspectes com a `terme`.
    
Troba la predicció de el model carregat per el sentiment d'aquest terme en aquesta frase. <br>
Notes: <br>
   - Pots ajudar-te dels resultats de la PRA2 per buscar frases. <br>
   - Pensa que també pots cercar la polaritat de diferents aspectes dins de la mateixa frase, pàg. ex. en la frase 'nice location but overpriced and poor service', hi ha dos aspectes: location i service. <br>
   - El model carregat ha estat entrenat amb una seqüència màxima de tokens de 80. <br>
</div>

## Anàlisi de sentiments per aspectes

In [31]:
#############################################
# SOLUCIÓ                                   #
#############################################
df = pd.read_csv('tripadvisor_data.csv')
#print(df.sample(20))
dfabsa= pd.DataFrame({'frase': df.iloc[[1,357,443,1098,1034,289,939,1367,400,200,1070,333]]['Short']})
dfabsa['terme']=['location']*12
dfabsa['label']=[2,1,2,1,2,2,2,1,1,1,2,2]
dfabsa.to_csv('dfabsa.csv')
print(dfabsa)

                                                  frase     terme  label
1     “Very nice food, great atmosphere, feels like ...  location      2
357                       “Overbooked and poor servive”  location      1
443                   “Lovely hotel fantastic location”  location      2
1098                                          “Just ok”  location      1
1034                   “Excellent pension and location”  location      2
289                  “Great location, hotel and owners”  location      2
939                          “Nice and quiet location.”  location      2
1367  “Very friendy family run hotel with great pool...  location      1
400                          “You get what you pay for”  location      1
200   “A great little hotel in an fantastic Italian ...  location      1
1070                       “Great value, good location”  location      2
333                    “Rooms small but great location”  location      2


In [32]:
TEXT = data.Field(sequential = False, use_vocab=False)
TERM = data.Field(sequential = False, use_vocab=False)
POLARITY = data.LabelField()

fields = [(None,None), ('text',TEXT), ('term',TERM), ('polarity',POLARITY)] 

train_data = data.TabularDataset(path = 'dfabsa.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True)

train_data, test_data = train_data.split(split_ratio = [0.6, 0.4], random_state = random.seed(SEED))

def tokenizar_e_indexar_con_bert(dataset, tokenizer):
    maxlen = 80
    for i in range(len(dataset.examples)):
        
        text = dataset.examples[i].text
        term = dataset.examples[i].term
        text_raw_sequence = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))
        term_raw_sequence = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(term))
        

        if len(term_raw_sequence)>=int(maxlen/4):
            term_raw_sequence = term_raw_sequence[0:int(maxlen/4)]
        seq = text_raw_sequence[:maxlen-3-len(term_raw_sequence)]

        text_bert_ids = tokenizer.convert_tokens_to_ids(["[CLS]"])\
            + seq +  tokenizer.convert_tokens_to_ids(["[SEP]"])\
            + term_raw_sequence + tokenizer.convert_tokens_to_ids(["[SEP]"])

        bert_segments_ids = [0] * (len(seq)+2) + [1] * (len(term_raw_sequence) + 1) + [0]*(maxlen-len(text_bert_ids))

        text_bert_ids = text_bert_ids + [0]*(maxlen-len(text_bert_ids))
        new_text = [text_bert_ids, bert_segments_ids]
        
        dataset.examples[i].text = np.asarray(text_bert_ids, dtype='int64')
        dataset.examples[i].term = np.asarray(bert_segments_ids, dtype='int64')

tokenizar_e_indexar_con_bert(train_data, tokenizer)
tokenizar_e_indexar_con_bert(test_data, tokenizer)

print(vars(train_data.examples[0]))
POLARITY.build_vocab(train_data)
print(POLARITY.vocab.freqs)

{'text': array([  101,  1523,  2200,  2767,  2100,  2155,  2448,  3309,  2007,
        2307,  4770,  1998, 12403,  4128,  1524,   102,  3295,   102,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0],
      dtype=int64), 'term': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64), 'polarity': '1'}
Counter({'2': 4, '1': 3})


In [33]:
BATCH_SIZE = 1

# si tenemos GPU disponible se computarán ahí los datos
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=False,
    device = device)

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train() 
    
    for batch in iterator:
        optimizer.zero_grad()  
        predictions = model(batch)   
        loss = criterion(predictions, batch.polarity.reshape(-1).long())        
        acc = categorical_accuracy(predictions, batch.polarity.reshape(-1).long())     
        loss.backward()     
        optimizer.step()      
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval() 
    
    with torch.no_grad(): 
    
        for batch in iterator:
            predictions = model(batch)     
            loss = criterion(predictions, batch.polarity.reshape(-1).long())
            acc = categorical_accuracy(predictions, batch.polarity.reshape(-1).long())
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

modelba   = modelba.to(device)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = torch.optim.Adam(modelba.parameters())

In [34]:
def categorical_accuracy(preds, y):
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y) # check if it is correct
    return correct.sum() / torch.FloatTensor([y.shape[0]])

N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(modelba, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(modelba, test_iterator, criterion)
    
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'absa.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Test Loss: {valid_loss:.3f} |  Test Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 1.764 | Train Acc: 28.57%
	 Test Loss: 0.755 |  Test Acc: 60.00%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 1.084 | Train Acc: 57.14%
	 Test Loss: 1.166 |  Test Acc: 40.00%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 2.342 | Train Acc: 28.57%
	 Test Loss: 2.670 |  Test Acc: 40.00%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 1.458 | Train Acc: 42.86%
	 Test Loss: 0.740 |  Test Acc: 60.00%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 0.988 | Train Acc: 42.86%
	 Test Loss: 0.793 |  Test Acc: 40.00%
